**Connect google drive**

In [1]:
# Check if NVIDIA GPU is enabled
!nvidia-smi

Thu Mar 17 16:01:24 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   31C    P8    28W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 android  'Colab Notebooks'   yolov3


**1) Clone the Darknet**



In [4]:
!git clone https://github.com/AlexeyAB/darknet

Cloning into 'darknet'...
remote: Enumerating objects: 15412, done.
remote: Total 15412 (delta 0), reused 0 (delta 0), pack-reused 15412
Receiving objects: 100% (15412/15412), 14.05 MiB | 17.19 MiB/s, done.
Resolving deltas: 100% (10354/10354), done.


# 새 섹션

**2) Compile Darknet using Nvidia GPU**


In [5]:
# change makefile to have GPU and OPENCV enabled
%cd darknet
!sed -i 's/OPENCV=0/OPENCV=1/' Makefile
!sed -i 's/GPU=0/GPU=1/' Makefile
!sed -i 's/CUDNN=0/CUDNN=1/' Makefile
!make

/content/darknet
mkdir -p ./obj/
mkdir -p backup
chmod +x *.sh
g++ -std=c++11 -std=c++11 -Iinclude/ -I3rdparty/stb/include -DOPENCV `pkg-config --cflags opencv4 2> /dev/null || pkg-config --cflags opencv` -DGPU -I/usr/local/cuda/include/ -DCUDNN -Wall -Wfatal-errors -Wno-unused-result -Wno-unknown-pragmas -fPIC -Ofast -DOPENCV -DGPU -DCUDNN -I/usr/local/cudnn/include -c ./src/image_opencv.cpp -o obj/image_opencv.o
./src/image_opencv.cpp: In function ‘void draw_detections_cv_v3(void**, detection*, int, float, char**, image**, int, int)’:
./src/image_opencv.cpp:946:23: warning: variable ‘rgb’ set but not used [-Wunused-but-set-variable]
                 float rgb[3];
                       ^~~
./src/image_opencv.cpp: In function ‘void draw_train_loss(char*, void**, int, float, float, int, int, float, int, char*, float, int, int, double)’:
./src/image_opencv.cpp:1147:13: warning: this ‘if’ clause does not guard... [-Wmisleading-indentation]
             if (iteration_old == 0)
           

**3) Configure Darknet network for training YOLO V3**

In [7]:
#!cp cfg/yolov3.cfg cfg/yolov3_training.cfg
!cp /mydrive/yolov3/data/yolov3_testing.cfg cfg/yolov3_training.cfg

In [8]:
!echo -e '' > data/obj.names
!echo -e '' > data/obj.data
!cp /mydrive/yolov3/data/obj.names data/obj.names
!cp /mydrive/yolov3/data/obj.data data/obj.data
!mkdir data/obj

In [9]:
# Download weights darknet model 53
!wget https://pjreddie.com/media/files/darknet53.conv.74

--2022-03-17 16:26:40--  https://pjreddie.com/media/files/darknet53.conv.74
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 162482580 (155M) [application/octet-stream]
Saving to: ‘darknet53.conv.74’

darknet53.conv.74   100%[===================>] 154.96M  19.9MB/s    in 8.8s    

2022-03-17 16:26:49 (17.6 MB/s) - ‘darknet53.conv.74’ saved [162482580/162482580]



**4) Extract Images**

The images need to be inside a zip archive called "images.zip" and they need to be inside the folder "yolov3" on Google Drive

In [10]:
!unzip /mydrive/yolov3/images.zip -d data/obj

Archive:  /mydrive/yolov3/images.zip
  inflating: data/obj/apple (1).jpg  
  inflating: data/obj/apple (1).txt  
  inflating: data/obj/apple (10).jpg  
  inflating: data/obj/apple (10).txt  
  inflating: data/obj/apple (2).jpg  
  inflating: data/obj/apple (2).txt  
  inflating: data/obj/apple (3).jpg  
  inflating: data/obj/apple (3).txt  
  inflating: data/obj/apple (4).jpg  
  inflating: data/obj/apple (4).txt  
  inflating: data/obj/apple (5).jpg  
  inflating: data/obj/apple (5).txt  
  inflating: data/obj/apple (6).jpg  
  inflating: data/obj/apple (6).txt  
  inflating: data/obj/apple (7).jpg  
  inflating: data/obj/apple (7).txt  
  inflating: data/obj/apple (8).jpg  
  inflating: data/obj/apple (8).txt  
  inflating: data/obj/apple (9).jpg  
  inflating: data/obj/apple (9).txt  
  inflating: data/obj/orange (1).jpg  
  inflating: data/obj/orange (1).txt  
  inflating: data/obj/orange (10).jpg  
  inflating: data/obj/orange (10).txt  
  inflating: data/obj/orange (2).jpg  
  in

In [11]:
import glob
images_list = glob.glob("data/obj/*.jpg")
#images_list = glob.glob("data/obj/*.png")
print(images_list)

['data/obj/orange (5).jpg', 'data/obj/apple (6).jpg', 'data/obj/orange (8).jpg', 'data/obj/orange (4).jpg', 'data/obj/orange (3).jpg', 'data/obj/apple (7).jpg', 'data/obj/apple (4).jpg', 'data/obj/apple (9).jpg', 'data/obj/apple (3).jpg', 'data/obj/apple (8).jpg', 'data/obj/orange (1).jpg', 'data/obj/orange (2).jpg', 'data/obj/apple (1).jpg', 'data/obj/orange (6).jpg', 'data/obj/apple (10).jpg', 'data/obj/orange (10).jpg', 'data/obj/orange (7).jpg', 'data/obj/orange (9).jpg', 'data/obj/apple (2).jpg', 'data/obj/apple (5).jpg']


In [12]:
#Create training.txt file
file = open("data/train.txt", "w") 
file.write("\n".join(images_list)) 
file.close() 

**6) Start the training**

In [ ]:
# Start the training
!./darknet detector train data/obj.data cfg/yolov3_training.cfg darknet53.conv.74 -dont_show

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
sh: 1: Syntax error: "(" unexpected

 Wrong annotation: class_id = 16. But class_id should be [from 0 to 1], file: data/obj/orange (1).txt 
sh: 1: Syntax error: "(" unexpected

 Wrong annotation: class_id = 15. But class_id should be [from 0 to 1], file: data/obj/apple (8).txt 
sh: 1: Syntax error: "(" unexpected
sh: 1: Syntax error: "(" unexpected

 Wrong annotation: class_id = 16. But class_id should be [from 0 to 1], file: data/obj/orange (1).txt 
sh: 1: Syntax error: "(" unexpected

 Wrong annotation: class_id = 15. But class_id should be [from 0 to 1], file: data/obj/apple (8).txt 
sh: 1: Syntax error: "(" unexpected

 Wrong annotation: class_id = 16. But class_id should be [from 0 to 1], file: data/obj/orange (1).txt 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Region 82 Avg (IOU: 0.000000), count: 1, class_loss = 0.000866, iou_loss = 0.000000, total_loss = 0.000866 
v3 (mse loss, Normalizer: (iou: 0.75, obj: 1.00, cls: 1.00) Re